In [1]:
#imports for institutional ownership
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen

#imports for alphavantage wrapper
#!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData 
ts = TimeSeries(key = 'KSTIKHKXANLCVJ2L', output_format = 'pandas', indexing_type = 'date')
ti = TechIndicators(key = 'KSTIKHKXANLCVJ2L', output_format = 'pandas', indexing_type = 'date')
fd = FundamentalData(key = 'KSTIKHKXANLCVJ2L', output_format = 'pandas', indexing_type = 'date')
key = 'HKDDDB1WO21X9QR6'
base_url = "https://www.alphavantage.co/query/"

#
from functools import reduce


In [2]:
ticker = 'DOCU'

In [3]:
#scapper - needs user:agent 

def get_institutional_ownership(ticker):
    values = []
    values_float = []
    columns = ['Institutional_Ownership', 'Change_Instituional_Ownership_3months']
    url = ("http://finviz.com/quote.ashx?t=" + ticker.lower())
    
    #use of user-agent to scape finviz from this post - https://stackoverflow.com/questions/54165551/scrape-finviz-page-for-specific-values-in-table
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = bs(webpage, "html.parser")
    
    #Grab institutional ownership 
    inst_ownership = soup.find(text = 'Inst Own').find_next(class_ = 'snapshot-td2').text
    values.append(inst_ownership)
    
    #Grab change in instiutional ownership
    change_inst_ownership_3mon = soup.find(text = 'Inst Trans').find_next(class_ = 'snapshot-td2').text
    values.append(change_inst_ownership_3mon)
    
    for value in values:
        value = value.strip('%')
        values_float.append(float(value)/100)
    df = pd.DataFrame(data = values_float, index = columns)
    df = df.transpose()
    return df

In [4]:
df = get_institutional_ownership(ticker)
df.head()

,Institutional_Ownership,Change_Instituional_Ownership_3months
0,0.753,0.0009


In [5]:
#Get ROE value from Alpha Vantage
data, meta_data = fd.get_company_overview(symbol = ticker)
df['ROE_TTM'] = float(data['ReturnOnEquityTTM'])

In [6]:
df

,Institutional_Ownership,Change_Instituional_Ownership_3months,ROE_TTM
0,0.753,0.0009,-0.4385


In [7]:
df.dtypes

Institutional_Ownership                  float64
Change_Instituional_Ownership_3months    float64
ROE_TTM                                  float64
dtype: object

In [8]:
#Import Data used to determine market direction (short term and long term trend)
data_150, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 150, series_type = 'close')
data_5, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 5, series_type = 'close')
data_10, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 10, series_type = 'close')
dfs = [data_5, data_10, data_150]

#merge three dataframes and pull last entry (which is the most relevant telling us the current market direction (short term and long))
#https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns
trend = reduce(lambda left, right: pd.merge(left, right, on='date', how = 'inner'), dfs)
trend.columns = ['IWO_5d_EMA', 'IWO_10d_EMA', 'IWO_150d_EMA']
trend = trend.iloc[[-1]]
trend

,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA
date,,,
2021-01-21,311.061,307.0653,248.0636


In [9]:
#Join all inforamtion used for CANSLIM screener into one DF and export to data folder

df = trend.join(df.set_index(trend.index))
df.head()

,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA,Institutional_Ownership,Change_Instituional_Ownership_3months,ROE_TTM
date,,,,,,
2021-01-21,311.061,307.0653,248.0636,0.753,0.0009,-0.4385


In [10]:
df.to_csv(f'data/{ticker}_CANSLIM_data.csv')